<a href="https://colab.research.google.com/github/soucs/dysarthric-asr/blob/main/wav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASR 1

In [ ]:
!pip install jiwer
!pip install transformers
!pip install librosa
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00


In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.36.0', '0.25.0')

In [ ]:
vocab_dict = {
              ' ': 21,
              "'": 13,
              'a': 24,
              'b': 17,
              'c': 25,
              'd': 2,
              'e': 9,
              'f': 14,
              'g': 22,
              'h': 8,
              'i': 4,
              'j': 18,
              'k': 5,
              'l': 16,
              'm': 6,
              'n': 7,
              'o': 10,
              'p': 19,
              'q': 3,
              'r': 20,
              's': 11,
              't': 0,
              'u': 26,
              'v': 27,
              'w': 1,
              'x': 23,
              'y': 15,
              'z': 12
             }
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

30


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
print(tokenizer)

Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	28: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	29: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	30: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	31: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
text = 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED'

import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(text_data):
    text_data = re.sub(chars_to_ignore_regex, '', text_data).lower()
    return text_data

audio_text = remove_special_characters(text)

In [ ]:
audio_text

'chapter sixteen i might have told you of the beginning of this liaison in a few lines but i wanted you to see every step by which we came i to agree to whatever marguerite wished'

## Speech signal

In [ ]:
import scipy.io.wavfile as wav
from scipy import signalload dataset
import numpy as np

# Define the desired sampling rate
desired_sampling_rate = 16000  # Change this to your desired rate

# Load the WAV file
file_path = '/content/audio1.wav'  # Replace with your file path
sampling_rate, audio_data = wav.read(file_path)

# Resample the audio signal to the desired sampling rate if it's different
if sampling_rate != desired_sampling_rate:
    audio_data_resampled = signal.resample(audio_data, int(len(audio_data) * (desired_sampling_rate / sampling_rate)))
    sampling_rate = desired_sampling_rate
else:
    audio_data_resampled = audio_data

# Convert the resampled audio signal to an array
audio_array = np.array(audio_data_resampled)

In [ ]:
audio_array.shape, audio_array.dtype

((232480,), dtype('int16'))

In [ ]:
def prepare_dataset(audio_array, audio_text):
    # batched output is "un-batched" to ensure mapping is correct
    inp_values = processor(audio_array, sampling_rate=16000).input_values[0]

    with processor.as_target_processor():
        target_values = processor(audio_text).input_ids
    return inp_values, target_values

In [ ]:
inp_values, target_values = prepare_dataset(audio_array, audio_text)

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
print(inp_values, len(target_values))

[ 0.01174984  0.01224541  0.01224541 ... -0.00410854 -0.00262181
 -0.00014394] 178


In [ ]:
feature = dict()
feature["input_values"] = inp_values
feature["labels"] = target_values
features = [feature]

## Trainer

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

class DataCollatorCTCWithPadding:
    # processor: Wav2Vec2Processor
    # padding: Union[bool, str] = True
    # max_length: Optional[int] = None
    # max_length_labels: Optional[int] = None
    # pad_to_multiple_of: Optional[int] = None
    # pad_to_multiple_of_labels: Optional[int] = None

    def __init__(self, processor, padding, max_length=1000000, max_length_labels=200, pad_to_multiple_of=2, pad_to_multiple_of_labels=2):
        self.processor = processor
        self.padding = padding
        self.max_length = max_length
        self.max_length_labels = max_length_labels
        self.pad_to_multiple_of = pad_to_multiple_of
        self.pad_to_multiple_of_labels = pad_to_multiple_of_labels

    def __call__(self, features: Dict[str, Union[List[int], torch.Tensor]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from jiwer import wer

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    word_er = wer(label_str, pred_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Stack of CNN layers for feature extraction
# Well pretrained. no fine tuning
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='/content/output',
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=features,
    eval_dataset=features,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more 

Step,Training Loss,Validation Loss


TrainOutput(global_step=30, training_loss=10.673201497395834, metrics={'train_runtime': 11.0364, 'train_samples_per_second': 2.718, 'train_steps_per_second': 2.718, 'total_flos': 3950146165248000.0, 'train_loss': 10.673201497395834, 'epoch': 30.0})

In [ ]:
def map_to_result(feature):
  with torch.no_grad():
    input_values = torch.tensor(feature["input_values"], device="cuda").unsqueeze(0)
    input_values = input_values.type(torch.cuda.HalfTensor)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  pred_str = processor.batch_decode(pred_ids)[0]
  text = processor.decode(feature["labels"], group_tokens=False)

  return pred_str, text

results = map_to_result(feature)


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
print(results[0])

qllelelvztkwx'lidvj</s>dtdvzfn</s>dtqdv[UNK]e<s><s></s>yyz</s>q qilq qq q dqdtdtdmtyetli dil i d  ial lczcqc</s>djzdkjdxxzdz</s>ziikdwjj<s>jxz</s>ivtixtzq<s>euopvv<s>jqv</s>[UNK]dxxzyqz<s></s>z<s>zttqeqertemtwetlqlqlivxj<s>zjzdzxdz</s>dtdxjddtieiedwkza '</s>'zxvzabdlzyxdevdvkdkzyei  lqq q qqelewjdt</s>dxzdvlvzdlddzxyxzkztxjqxjlidz</s> zyk</s>dttqvve


# ASR 2

In [ ]:
!pip install datasets
!pip install librosa
!pip install jiwer

In [ ]:
!pip install accelerate -U
!pip install transformers -U

import accelerate
import transformers

transformers.__version__, accelerate.__version__

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
data = load_dataset("lj_speech")

Generating train split:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [ ]:
data = data['train'].train_test_split(0.2)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'file', 'text', 'normalized_text'],
        num_rows: 10480
    })
    test: Dataset({
        features: ['id', 'audio', 'file', 'text', 'normalized_text'],
        num_rows: 2620
    })
})

In [ ]:
data['train'][0]

{'id': 'LJ009-0265',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ009-0265.wav',
  'array': array([ 0.00000000e+00,  1.22070312e-04,  1.22070312e-04, ...,
         -6.10351562e-05, -3.35693359e-04, -7.62939453e-04]),
  'sampling_rate': 22050},
 'file': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ009-0265.wav',
 'text': 'Even now his sufferings were not at an end,',
 'normalized_text': 'Even now his sufferings were not at an end,'}

## Create Wav2Vec2Processor

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

data = data.map(remove_special_characters)

Map:   0%|          | 0/10480 [00:00<?, ? examples/s]

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

vocab_dict

Map:   0%|          | 0/10480 [00:00<?, ? examples/s]

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

{'w': 0,
 'r': 1,
 ']': 2,
 'f': 3,
 'é': 4,
 '1': 5,
 '2': 6,
 'p': 7,
 'c': 8,
 '3': 9,
 '’': 10,
 '&': 11,
 '[': 12,
 'v': 13,
 '5': 14,
 'j': 16,
 '0': 17,
 'e': 18,
 'i': 19,
 "'": 20,
 ')': 21,
 'è': 22,
 '“': 23,
 '9': 24,
 '6': 25,
 'l': 26,
 'm': 27,
 'à': 28,
 'q': 29,
 '$': 30,
 'd': 31,
 '7': 32,
 'x': 33,
 'n': 34,
 'ü': 35,
 'y': 36,
 'a': 37,
 '4': 38,
 '”': 39,
 'h': 40,
 't': 41,
 'ê': 42,
 'g': 43,
 'u': 44,
 'b': 45,
 'k': 46,
 '8': 47,
 'z': 48,
 'o': 49,
 'â': 50,
 '£': 51,
 's': 52,
 '(': 53,
 '|': 15,
 '[UNK]': 54,
 '[PAD]': 55}

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

## Preprocess Speech Data

In [ ]:
from datasets import load_dataset
import librosa
import torchaudio

# # Specify the target sampling rate
# target_sampling_rate = 16000  # Adjust this to your desired sampling rate

# # Function to resample an audio waveform to the target sampling rate
# def resample_audio(batch):
#     waveform, sample_rate = torchaudio.load(batch['file'])  # Assuming 'file' is the key for audio paths in your dataset
#     resampled_waveform = librosa.resample(waveform.numpy()[0], orig_sr=sample_rate, target_sr=target_sampling_rate)
#     batch['audio']['array'] = resampled_waveform
#     batch['audio']['sampling_rate'] = target_sampling_rate
#     return batch

# # Apply the resampling function to all batches in the dataset
# data = data.map(resample_audio)

# # Access the resampled data
# print(data)

In [ ]:
# Specify the target sampling rate
target_sampling_rate = 16000  # Adjust this to your desired sampling rate

# Function to resample an audio waveform to the target sampling rate
def resample_audio(batch):
    waveform, sample_rate = torchaudio.load(batch['file'])  # Assuming 'file' is the key for audio paths in your dataset
    resampled_waveform = librosa.resample(waveform.numpy()[0], orig_sr=sample_rate, target_sr=target_sampling_rate)
    batch['resamp_array'] = resampled_waveform
    return batch

# Apply the resampling function to all batches in the dataset
data = data.map(resample_audio)

Map:   0%|          | 0/10480 [00:00<?, ? examples/s]

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

In [ ]:
# def prepare_dataset(batch):
#     audio = batch["audio"]

#     # batched output is "un-batched" to ensure mapping is correct
#     batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

#     with processor.as_target_processor():
#         batch["labels"] = processor(batch["text"]).input_ids
#     return batch

def prepare_dataset(batch):
    # batched output is "un-batched" to ensure mapping is correct
    batch["resamp_array"] = processor(batch["resamp_array"], sampling_rate=16000).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
data = data.map(prepare_dataset, remove_columns=data.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/10480 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/2620 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

## Train and Evaluate

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

wer_metric = load_metric("wer")

<ipython-input-16-b7061bd19f45>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='/content/output',
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

ImportError: ignored

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    tokenizer=processor.feature_extractor,
)

NameError: ignored

In [ ]:
trainer.train()